In [16]:
# Necessary imports
%load_ext autoreload
%autoreload 2\

import networkx as nx
import numpy as np
import tensorflow as tf

from keras import backend as K
from keras.layers import Dense


from TCGAMultiOmics.multiomics import MultiOmicsData
from moge.network.heterogeneous_network import HeterogeneousNetwork


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


#  Import network from file

In [2]:
import pickle

# WRITE
# with open('moge/data/lncRNA_miRNA_mRNA/miRNA-mRNA_network_test_05_val_01_seed_0.pickle', 'wb') as file:
#     pickle.dump(network, file)

# READ
with open('moge/data/lncRNA_miRNA_mRNA/miRNA-mRNA_network_test_05_val_01_seed_0.pickle', 'rb') as file:
# with open('moge/data/lncRNA_miRNA_mRNA/miRNA-mRNA_network.pickle', 'rb') as file:
    network = pickle.load(file)
    network.remove_extra_nodes()
    network.node_list = network.all_nodes
#     node_list = network.node_list

In [3]:
# READ edgelists
with open('moge/data/lncRNA_miRNA_mRNA/miRNA-mRNA_network_test_05_val_01_seed_0_test_edges.pickle', 'rb') as file:
    test_edges_dict = pickle.load(file)
    
with open('moge/data/lncRNA_miRNA_mRNA/miRNA-mRNA_network_test_05_val_01_seed_0_val_edges.pickle', 'rb') as file:
    val_edges_dict = pickle.load(file)

# Load training data

In [5]:
X, y = network.multi_omics_data.load_data(modalities=["MIR", "GE"])

In [12]:
X["MIR"].shape

(460, 1870)

In [13]:
node_features_size = X["MIR"].shape[0]

In [19]:
_d = 128

In [20]:
n_nodes = len(network.node_list)

# Training Source Target Graph Embedding

In [39]:
tf.reset_default_graph()
sess.close()
sess = tf.InteractiveSession()

In [40]:
with tf.name_scope('inputs'):
    E_ij = tf.placeholder(tf.float32, shape=(1,), name="E_ij")
    N_i = tf.placeholder(tf.float32, shape=(1, node_features_size), name="N_i")
    N_j = tf.placeholder(tf.float32, shape=(1, node_features_size), name="N_j")
    is_directed = tf.placeholder(tf.bool, name="is_directed")
    i = tf.Variable(int, name="i", trainable=False)
    j = tf.Variable(int, name="j", trainable=False)

In [41]:
with tf.name_scope('siamese'):
    emb_c_i = Dense(128, activation='relu')(N_i)
    emb_c_i = Dense(_d, activation='relu')(emb_c_i)

    emb_c_j = Dense(128, activation='relu')(N_j)
    emb_c_j = Dense(_d, activation='relu')(emb_c_j)

In [45]:
with tf.name_scope('embedding'):
    emb_s = tf.Variable(initial_value=tf.random_uniform([n_nodes, int(_d/2)], -1, 1),
                        validate_shape=True, dtype=tf.float32,
                        name="emb_s", trainable=True)

    emb_t = tf.Variable(initial_value=tf.random_uniform([n_nodes, int(_d/2)], -1, 1),
                        validate_shape=True, dtype=tf.float32,
                        name="emb_s", trainable=True)

    emb_c = tf.concat([emb_s, emb_t], axis=1, name="emb_concat")

In [50]:
emb_c[i, :]

<tf.Tensor 'strided_slice_3:0' shape=(128,) dtype=float32>

In [49]:
emb_c[i] = emb_c[i, :].assign(emb_c_i)

ValueError: Sliced assignment is only supported for variables

In [51]:
emb_c_i

<tf.Tensor 'siamese/dense_2/Relu:0' shape=(1, 128) dtype=float32>

In [ ]:
sess.as_default()
K.set_session(sess)
session.run(init_op)